# Part One

Import libraries:

In [1]:
conda install -c conda-forge folium

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import requests
import io
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
from geopy.geocoders import Nominatim

Read the Wiki table into a pandas dataframe:

In [4]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Drop the 'not assigned' Boroughs from the df:

In [5]:
df.drop(df.loc[df['Borough']=='Not assigned'].index, inplace=True)
df

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


If the postal codes have more than one neighbourhood this combines them in one row and separates with a comma:

In [6]:
torontogrouped = df.groupby(['Postal Code','Borough'], as_index=False).agg(lambda x: ','.join(x))
torontogrouped

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


There are no no rows with a Borough but a 'Not assigned' neighbourhood. Therefore, nothing process needed to remove these.

In [7]:
torontogrouped.shape

(103, 3)

# Part Two

Getting the coordinates for Toronto postcodes:

In [8]:
df_coord = pd.read_csv('http://cocl.us/Geospatial_data')
df_coord

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


Merging the two dataframes:

In [9]:
#torontogrouped = torontogrouped.rename(columns={"State or union territory": "State","Population(2011)[3]": "Population"})
df_toronto = pd.merge(torontogrouped,df_coord,on = 'Postal Code')
df_toronto

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


# Part Three

Foursquare credentials:

In [10]:
CLIENT_ID = 'ULRY2I11ZLLTODP1AJONCCEUTZZCEYFWEAXFE2CTWTMCTS1K' 
CLIENT_SECRET = 'APYM3EIBIRXESZCEO4EN5XUDZVF5ZHOJXW34M1VHGZRYDYD5' 
VERSION = '20180605'
LIMIT = 100
radius = 500

Foursquare API to get most common venues:

In [11]:
#Function to apply it to every Postcode we have.
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

In [13]:
print(toronto_venues.shape)
toronto_venues.head()

(2113, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant


In [14]:
toronto_venues.groupby('Neighbourhood').count().head()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23


One Hot encoding each unique venue category:

In [15]:
# using one hot encoding for the different venue categories
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

#adding the Neighbourhoods column from the toronto_venues dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood']

#reordering the toronto_onehot dataframe to have Neighbourhoods as the first column
list_columns = toronto_onehot.columns.tolist()
list_columns.remove('Neighbourhood')
new_list=['Neighbourhood']
new_list.extend(list_columns)
toronto_onehot = toronto_onehot[new_list]
toronto_onehot.head()

,Neighbourhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
toronto_onehot_grouped = toronto_onehot.groupby(['Neighbourhood']).mean().reset_index()
toronto_onehot_grouped.head()

,Neighbourhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Finding most common venues for each neighbourhood:

In [17]:
#printing top 5 most common venues for each neighbourhood
for ngh in toronto_onehot_grouped['Neighbourhood']:
    temp = toronto_onehot_grouped[toronto_onehot_grouped['Neighbourhood'] == ngh].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})

Dropping neighbourhoods having less than 5 most common venues:

In [18]:
toronto_onehot_grouped_sum = toronto_onehot.groupby(['Neighbourhood']).sum().reset_index()
toronto_sum = toronto_onehot_grouped_sum.sum(axis=1)
toronto_onehot_grouped_sum = toronto_onehot_grouped_sum.loc[toronto_sum >= 5,:]
toronto_onehot_grouped_sum.reset_index(drop = True)
toronto_onehot_grouped.set_index('Neighbourhood',inplace=True)
toronto_onehot_grouped = toronto_onehot_grouped.loc[toronto_onehot_grouped_sum['Neighbourhood'],:]

In [19]:
toronto_onehot_grouped.reset_index(inplace=True)

Creating DF with top common venues for each neighbourhood:

In [20]:
#function to return num_top_venues most common venues for a neighborhood
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
#storing the top 5 toronto corresponding to each Neighbourhood in a datframe
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top toronto
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Neighbourhoods_toronto_sorted = pd.DataFrame(columns=columns)
Neighbourhoods_toronto_sorted['Neighbourhood'] = toronto_onehot_grouped['Neighbourhood']

for ind in np.arange(toronto_onehot_grouped.shape[0]):
    Neighbourhoods_toronto_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_onehot_grouped.iloc[ind, :], num_top_venues)

Neighbourhoods_toronto_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Alderwood, Long Branch",Pizza Place,Pharmacy,Skating Rink,Coffee Shop,Pub
1,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Sandwich Place,Sushi Restaurant,Intersection
2,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Italian Restaurant,Thai Restaurant,Locksmith
3,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Restaurant
4,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Yoga Studio,Stadium


Using K-means to cluster 3 groups:

In [22]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_onehot_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([2, 2, 2, 0, 0, 2, 0, 0, 1, 0, 1, 0, 2, 0, 2, 2, 2, 0, 2, 1, 1, 2,
       0, 0, 0, 2, 1, 0, 0, 1, 1, 2, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 2,
       0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 0, 2, 2, 2, 2, 1])

In [23]:
# add clustering labels
Neighbourhoods_toronto_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(Neighbourhoods_toronto_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,NaN,NaN,NaN,NaN,NaN,NaN
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,NaN,NaN,NaN,NaN,NaN,NaN
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0.0,Mexican Restaurant,Electronics Store,Breakfast Spot,Medical Center,Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,NaN,NaN,NaN,NaN,NaN,NaN
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Lounge,Gas Station,Bank,Caribbean Restaurant,Athletics & Sports


Making the 'Cluster Labels' integers so they can be mapped:

In [24]:
toronto_merged.shape
toronto_merged = toronto_merged.dropna(axis='rows')
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
toronto_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,Mexican Restaurant,Electronics Store,Breakfast Spot,Medical Center,Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,Lounge,Gas Station,Bank,Caribbean Restaurant,Athletics & Sports
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,0,Discount Store,Bus Station,Department Store,Coffee Shop,Chinese Restaurant
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577,1,Bakery,Park,Metro Station,Ice Cream Shop,Bus Station
10,M1P,Scarborough,"Dorset Park, Wexford Heights, Scarborough Town...",43.757410,-79.273304,2,Indian Restaurant,Pet Store,Vietnamese Restaurant,Brewery,Chinese Restaurant


In [25]:
#fetching the coordinates of Toronto using the geopy library
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Cluster Map:

In [39]:
#visualising the above clusters using folium on the map of toronto
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(2)
colors_array = cm.rainbow(np.linspace(0, 1, len(x)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [27]:
#Cluster 0 (Red color)
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,Scarborough,0,Mexican Restaurant,Electronics Store,Breakfast Spot,Medical Center,Restaurant
6,Scarborough,0,Discount Store,Bus Station,Department Store,Coffee Shop,Chinese Restaurant
18,North York,0,Clothing Store,Coffee Shop,Fast Food Restaurant,Restaurant,Japanese Restaurant
26,North York,0,Gym,Japanese Restaurant,Restaurant,Beer Store,Coffee Shop
27,North York,0,Gym,Japanese Restaurant,Restaurant,Beer Store,Coffee Shop
29,North York,0,Falafel Restaurant,Vietnamese Restaurant,Caribbean Restaurant,Bar,Coffee Shop
37,East Toronto,0,Trail,Asian Restaurant,Health Food Store,Neighborhood,Pub
38,East York,0,Coffee Shop,Sporting Goods Shop,Burger Joint,Shopping Mall,Bank
41,East Toronto,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop
43,East Toronto,0,Coffee Shop,Bakery,Brewery,Gastropub,American Restaurant


In [28]:
#Cluster 1 (Purple Color)
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,Scarborough,1,Lounge,Gas Station,Bank,Caribbean Restaurant,Athletics & Sports
7,Scarborough,1,Bakery,Park,Metro Station,Ice Cream Shop,Bus Station
17,North York,1,Fast Food Restaurant,Athletics & Sports,Mediterranean Restaurant,Golf Course,Pool
30,North York,1,Park,Grocery Store,Business Service,Bank,Baseball Field
31,North York,1,Park,Grocery Store,Business Service,Bank,Baseball Field
32,North York,1,Park,Grocery Store,Business Service,Bank,Baseball Field
33,North York,1,Park,Grocery Store,Business Service,Bank,Baseball Field
36,East York,1,Park,Beer Store,Athletics & Sports,Skating Rink,Curling Ice
75,Downtown Toronto,1,Grocery Store,Café,Park,Candy Store,Coffee Shop
76,West Toronto,1,Bakery,Pharmacy,Pet Store,Liquor Store,Middle Eastern Restaurant


In [29]:
#Cluster 2 (orange color)
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
10,Scarborough,2,Indian Restaurant,Pet Store,Vietnamese Restaurant,Brewery,Chinese Restaurant
11,Scarborough,2,Auto Garage,Bakery,Sandwich Place,Shopping Mall,Middle Eastern Restaurant
13,Scarborough,2,Fast Food Restaurant,Pizza Place,Gas Station,Noodle House,Chinese Restaurant
15,Scarborough,2,Fast Food Restaurant,Pharmacy,Breakfast Spot,Intersection,Pizza Place
22,North York,2,Ramen Restaurant,Sushi Restaurant,Café,Pizza Place,Coffee Shop
24,North York,2,Pizza Place,Butcher,Coffee Shop,Pharmacy,Grocery Store
28,North York,2,Coffee Shop,Bank,Sandwich Place,Sushi Restaurant,Intersection
35,East York,2,Pizza Place,Pharmacy,Gastropub,Intersection,Café
39,East York,2,Indian Restaurant,Sandwich Place,Yoga Studio,Pizza Place,Bank
42,East Toronto,2,Fast Food Restaurant,Ice Cream Shop,Sandwich Place,Italian Restaurant,Fish & Chips Shop
